## 1000 Random Articles

This sample was gathered for coding

In [1]:
! pip install openpyxl

In [2]:
import os
import pandas as pd
import openpyxl
from datetime import datetime
import matplotlib.pyplot as plt
from textblob import TextBlob
import ast
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
from nltk.tokenize import word_tokenize
from collections import Counter
from bertopic import BERTopic
import random
import numpy as np
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import matplotlib.cm as cm
from collections import Counter
from umap import UMAP
from wordcloud import WordCloud
import seaborn as sns
from scipy.stats import f_oneway
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from gensim import corpora, models
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import linkage, fcluster

In [3]:
# Load the Excel file into a DataFrame
data_updated = '/Users/helgegeurtjacobusmoes/Desktop/thesis data/Updated_Data_And_Topics.xlsx'

# Load the Excel file into a DataFrame
data_updated = pd.read_excel(data_updated)


In [4]:
data_updated

,Headline,Publication,URL,News Outlet,Type of News,Word Count,Body,Publication Date,Combined,Topic,Probabilities,Topic Name
0,"Nee, kunstmatige intelligentie gaat ons niet u...","Trouw, Verdieping; Blz. 4, 5, 2044 words",https://advance.lexis.com/api/document?collect...,Trouw,Verdieping,2044,Welkom in de AI-fabriek serie\nDat kunstmatige...,07-12-2023,nee kunstmatige intelligentie gaat uitroeien w...,0,0.716202,Technology
1,Wereldleiders zoeken grip op kunstmatige intel...,"Trouw, Vandaag; Blz. 6, 528 words",https://advance.lexis.com/api/document?collect...,Trouw,Vandaag,528,Op het Britse landgoed Bletchley Park werden t...,03-11-2023,wereldleiders zoeken grip kunstmatige intellig...,1,1.000000,Politics
2,Kunstmatige intelligentie is best bedreigend,"Trouw, Tijdgeest; Blz. 8, 576 words",https://advance.lexis.com/api/document?collect...,Trouw,Tijdgeest,576,Of kunstmatige intelligentie nuttig is (Tijdge...,13-05-2023,kunstmatige intelligentie best bedreigend kuns...,-1,0.000000,AI in Society
3,Mensen zijn een stuk efficiënter dan kunstmati...,"Trouw, Vandaag; Blz. 3, 741 words",https://advance.lexis.com/api/document?collect...,Trouw,Vandaag,741,De wereld raakte het afgelopen jaar in de ban ...,21-10-2023,mensen stuk efficiënter kunstmatige intelligen...,-1,0.000000,AI in Society
4,Bedreigt kunstmatige intelligentie ons godsbeeld?,"Trouw, Religie en Filosofie; Blz. 8, 9, 1367 w...",https://advance.lexis.com/api/document?collect...,Trouw,Religie en Filosofie,1367,Theologisch elftal\n'In het begin was het Woor...,16-12-2022,bedreigt kunstmatige intelligentie godsbeeld t...,8,1.000000,Education
...,...,...,...,...,...,...,...,...,...,...,...,...
6441,De rauwe realiteit,"Het Financieele Dagblad, MORGEN; Blz. 4, 2920 ...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,MORGEN,2920,Grootse oplossingen\nDrie stedelijke 'ontwrich...,14-10-2017,rauwe realiteit grootse oplossingen drie stede...,-1,0.000000,AI in Society
6442,No Headline In Original,"Het Financieele Dagblad, PAGINA 13; Blz. 13, 1...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,PAGINA,114,"klinkt als muziek\nDe Walkman, van Sony, is vo...",29-04-2023,no headline original klinkt muziek walkman son...,-1,0.000000,AI in Society
6443,Groeten uit het hart van de hightech,"Het Financieele Dagblad, WEEKEND; Blz. 6, 2799...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,WEEKEND,2799,Het is zover voor 'onze man in San Francisco'....,20-08-2016,groeten hart hightech zover man san francisco ...,-1,0.000000,AI in Society
6444,De complete lijst Jonge Talenten 2019,"Het Financieele Dagblad, FD PERSOONLIJK; Arbei...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,FD PERSOONLIJK; Arbeidsmarkt,8007,Rebel werkte zes jaar bij zakenbank Morgan Sta...,17-01-2019,complete lijst jonge talenten rebel werkte z j...,1,0.880587,Politics


In [5]:
# Redefine the balance_news_outlets function
def balance_news_outlets(topic_data, n_samples=100):
    news_outlets = topic_data['News Outlet'].unique()
    samples_per_outlet = n_samples // len(news_outlets)
    remainder = n_samples % len(news_outlets)
    
    balanced_sample = pd.DataFrame()
    
    for outlet in news_outlets:
        outlet_data = topic_data[topic_data['News Outlet'] == outlet]
        outlet_sample_size = min(samples_per_outlet, len(outlet_data))
        if outlet_sample_size > 0:
            outlet_sample = outlet_data.sample(n=outlet_sample_size, random_state=1, replace=True)
            balanced_sample = pd.concat([balanced_sample, outlet_sample])
    
    if remainder > 0:
        additional_sample = topic_data.sample(n=remainder, random_state=1, replace=True)
        balanced_sample = pd.concat([balanced_sample, additional_sample])
    
    return balanced_sample.sample(n=n_samples, random_state=1, replace=True)  # Ensure final sample size is exactly n_samples

# Re-filter the dataset to include only the Dutch articles, topics 0-10, and non-NaN publication dates
dutch_articles = data_updated[(data_updated['Topic'].between(0, 10)) & 
                              (data_updated['News Outlet'].notnull()) & 
                              (data_updated['Publication Date'].notna())]

# Initialize an empty dataframe to hold the final samples
final_dutch_samples_corrected = pd.DataFrame()

# Sample 100 rows from each topic, and balance the news outlet representation
for topic in range(0, 11):
    topic_data = dutch_articles[dutch_articles['Topic'] == topic]
    if not topic_data.empty:
        balanced_topic_sample = balance_news_outlets(topic_data, n_samples=100)
        final_dutch_samples_corrected = pd.concat([final_dutch_samples_corrected, balanced_topic_sample])

# Check if we have exactly 1000 samples, if not, add one more sample
if len(final_dutch_samples_corrected) < 1001:
    additional_sample = dutch_articles.sample(n=1, random_state=1, replace=True)
    final_dutch_samples_corrected = pd.concat([final_dutch_samples_corrected, additional_sample])

# Reset index for the final_samples dataframe
final_dutch_samples_corrected.reset_index(drop=True, inplace=True)

# Display the first few rows of the final samples
final_dutch_samples_corrected

,Headline,Publication,URL,News Outlet,Type of News,Word Count,Body,Publication Date,Combined,Topic,Probabilities,Topic Name
0,In dit lab lijkt alles echter dan levensecht,"AD/Algemeen Dagblad, Z Weekend; Blz. 10, 11, 1...",https://advance.lexis.com/api/document?collect...,AD/Algemeen Dagblad,Z Weekend,1097,Werkelijkheid nagebootst bij Tilburg Universit...,09-09-2023,lab lijkt echter levensecht werkelijkheid nage...,0,1.000000,Technology
1,Slimme kinderen denken over slimme robots,"Trouw, Vandaag; Blz. 2, 233 words",https://advance.lexis.com/api/document?collect...,Trouw,Vandaag,233,"""Als een robot gemaakt is om mijn vriend te zi...",24-05-2023,slimme kinderen denken slimme robot robot gema...,0,1.000000,Technology
2,Geestelijk vader vande robotgodin,"NRC.NEXT, ECONOMIE; Blz. 5, 239 words",https://advance.lexis.com/api/document?collect...,NRC,ECONOMIE,239,Chen Xiaoping (1955)\nDeze hoogleraar kunstmat...,17-09-2020,geestelijk vader vande robotgodin chen xiaopin...,0,0.756673,Technology
3,Kunstmatige intelligentie begint gevaarlijk te...,"Trouw, Buitenland; Blz. 12, 399 words",https://advance.lexis.com/api/document?collect...,Trouw,Buitenland,399,Brandbrief\nHet moment om tijdelijk te stoppen...,30-03-2023,kunstmatige intelligentie begint gevaarlijk al...,0,0.808183,Technology
4,Tesla -topman: pas op met onze zelfrijdfunctie,"NRC.NEXT, economie, 484 words",https://advance.lexis.com/api/document?collect...,NRC,economie,484,Zelfrijdende auto'sTesla-auto's zijn sinds gis...,16-10-2015,tesla pa onze zelfrijdfunctie zelfrijdende sin...,0,1.000000,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...
996,Basilicum kan ook de hoogte in,"de Volkskrant, Ten Eerste; Blz. 15, 89 words",https://advance.lexis.com/api/document?collect...,de Volkskrant,Ten Eerste,89,Horticultuur\nBezoekers bekijken een stellage ...,12-06-2019,basilicum hoogte horticultuur bezoekers bekijk...,9,0.700835,Environment
997,Pas naar de hangar als iets bijna kapot is,"NRC, WETENSCHAP; Blz. 16, 987 words",https://advance.lexis.com/api/document?collect...,NRC,WETENSCHAP,987,Reportage Kunstmatige intelligentie Veiligheid...,03-06-2022,pa hangar bijna kapot reportage kunstmatige in...,9,0.911220,Environment
998,"Red jezelf,geef de giraf een portemonnee Sanne...","NRC.NEXT, OPINIE & DEBAT; Blz. 1, 932 words",https://advance.lexis.com/api/document?collect...,NRC,OPINIE & DEBAT,932,Opinie\nNatuur\nDe natuur is er niet om te exp...,11-07-2020,red jezelf geef giraf portemonnee sanne bloemi...,9,0.602084,Environment
999,De veleleefgeluiden van onder de zeespiegel,"Trouw, Duurzaamheid en Natuur; Blz. 14, 15, 13...",https://advance.lexis.com/api/document?collect...,Trouw,Duurzaamheid en Natuur,1355,"Zeeleven interview\nWalvissen zingen, orka's k...",19-03-2022,veleleefgeluiden zeespiegel zeeleven interview...,9,0.520151,Environment


In [6]:
# Check for duplicates and count them
duplicates = final_dutch_samples_corrected.duplicated()
num_duplicates = duplicates.sum()

print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 378


In [7]:
# Redefine the balance_news_outlets function
def balance_news_outlets(topic_data, n_samples=100):
    news_outlets = topic_data['News Outlet'].unique()
    samples_per_outlet = n_samples // len(news_outlets)
    remainder = n_samples % len(news_outlets)
    
    balanced_sample = pd.DataFrame()
    
    for outlet in news_outlets:
        outlet_data = topic_data[topic_data['News Outlet'] == outlet]
        outlet_sample_size = min(samples_per_outlet, len(outlet_data))
        if outlet_sample_size > 0:
            outlet_sample = outlet_data.sample(n=outlet_sample_size, random_state=1, replace=False)
            balanced_sample = pd.concat([balanced_sample, outlet_sample])
    
    if remainder > 0:
        available_samples = len(topic_data) - len(balanced_sample)
        remainder_sample_size = min(remainder, available_samples)
        additional_sample = topic_data.drop(balanced_sample.index).sample(n=remainder_sample_size, random_state=1, replace=False)
        balanced_sample = pd.concat([balanced_sample, additional_sample])
    
    # Ensure we do not attempt to sample more than the available data
    final_sample_size = min(n_samples, len(balanced_sample))
    return balanced_sample.sample(n=final_sample_size, random_state=1, replace=False)

# Re-filter the dataset to include only the Dutch articles, topics 0-10, and non-NaN publication dates, excluding topic -1
dutch_articles = data_updated[(data_updated['Topic'].between(0, 10)) & 
                              (data_updated['News Outlet'].notnull()) & 
                              (data_updated['Publication Date'].notna())]

# Initialize an empty dataframe to hold the final samples
final_dutch_samples_corrected = pd.DataFrame()

# Sample 100 rows from each topic, and balance the news outlet representation
for topic in range(0, 11):
    topic_data = dutch_articles[dutch_articles['Topic'] == topic]
    if not topic_data.empty:
        balanced_topic_sample = balance_news_outlets(topic_data, n_samples=100)
        final_dutch_samples_corrected = pd.concat([final_dutch_samples_corrected, balanced_topic_sample])

# Ensure the final sample size is exactly 1000 and remove duplicates
final_dutch_samples_corrected = final_dutch_samples_corrected.drop_duplicates()
final_sample_size = min(1000, len(final_dutch_samples_corrected))
final_dutch_samples_corrected = final_dutch_samples_corrected.sample(n=final_sample_size, random_state=1, replace=False)

# Add additional articles from the dataframe to ensure we have 1000 unique articles, excluding topic -1
if len(final_dutch_samples_corrected) < 1001:
    additional_needed = 1001 - len(final_dutch_samples_corrected)
    additional_articles = data_updated[
        (data_updated['Topic'].between(0, 10)) &
        (data_updated['News Outlet'].notnull()) &
        (data_updated['Publication Date'].notna()) &
        (~data_updated.index.isin(final_dutch_samples_corrected.index))
    ].sample(n=additional_needed, random_state=1, replace=False)
    final_dutch_samples_corrected = pd.concat([final_dutch_samples_corrected, additional_articles]).drop_duplicates().sample(n=1001, random_state=1, replace=False)

# Reset index for the final_samples dataframe
final_dutch_samples_corrected.reset_index(drop=True, inplace=True)

# Group by topic and sort the final samples
final_dutch_samples_corrected = final_dutch_samples_corrected.sort_values(by='Topic').reset_index(drop=True)

# Display the first few rows of the final samples
final_dutch_samples_corrected


,Headline,Publication,URL,News Outlet,Type of News,Word Count,Body,Publication Date,Combined,Topic,Probabilities,Topic Name
0,Zelfrijdende auto binnen vijf jaar in de stad',"AD/Algemeen Dagblad, Auto; Blz. 26, 27, 744 words",https://advance.lexis.com/api/document?collect...,AD/Algemeen Dagblad,Auto,744,Computer wordt perfecte bestuurder\nBijna alle...,15-04-2016,zelfrijdende auto binnen vijf jaar stad comput...,0,1.000000,Technology
1,Van oermens tot kunstmatige intelligentie 2020...,"NRC.NEXT, WETENSCHAP; Blz. 1, 1640 words",https://advance.lexis.com/api/document?collect...,NRC,WETENSCHAP,1640,Vooruitblik 2020 Wat kunnen we verwachten van ...,04-01-2020,oermens kunstmatige intelligentie klimaat cris...,0,0.824260,Technology
2,"De technologie gaat zo hard, dat een robot ove...","Het Financieele Dagblad, NIEUWS; Blz. 20, 984 ...",https://advance.lexis.com/api/document?collect...,Het Financieele Dagblad,NIEUWS,984,GEKKOGRIJPERS\nVan onze redacteur\nDallas\nRob...,26-07-2019,technologie gaat hard robot drie jaar bijna ge...,0,1.000000,Technology
3,Robot bij verkoopgesprek,"De Telegraaf, FINANCIEEL; Blz. 24, 519 words",https://advance.lexis.com/api/document?collect...,De Telegraaf,FINANCIEEL,519,Expert ’Vanaf 2025 gaan computers met elkaar o...,13-02-2019,robot verkoopgesprek expert vanaf gaan compute...,0,1.000000,Technology
4,Wondere wereld op wielen Kunstmatige intellige...,"De Telegraaf, BINNENLAND; Blz. 7, 821 words",https://advance.lexis.com/api/document?collect...,De Telegraaf,BINNENLAND,821,Kunstmatige intelligentie domineert grootste t...,11-01-2024,wondere wereld wielen kunstmatige intelligenti...,0,1.000000,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...
996,"Nee, uw kat mist u echt niet","Trouw, Religie en Filosofie; Blz. 6, 7, 1275 w...",https://advance.lexis.com/api/document?collect...,Trouw,Religie en Filosofie,1275,Dierenfilosofie interview\nEen filosofische re...,11-09-2023,nee kat mist echt dierenfilosofie interview fi...,9,0.586878,Environment
997,Het bestaat echt: een slim vogelhuisje getest,"De Telegraaf, FINANCIEEL; Blz. 26, 386 words",https://advance.lexis.com/api/document?collect...,De Telegraaf,FINANCIEEL,386,getest\nProduct: Bird BuddyPrijs: €215\nHet wa...,05-05-2023,bestaat echt slim vogelhuisje getest getest pr...,9,1.000000,Environment
998,Hoge Veluwe stopt met wildkiekjes,"De Telegraaf, BINNENLAND; Blz. 5, 248 words",https://advance.lexis.com/api/document?collect...,De Telegraaf,BINNENLAND,248,OTTERLO - Het Nationale Park De Hoge Veluwe st...,08-01-2022,hoge veluwe stopt wildkiekjes otterlo national...,9,0.998493,Environment
999,Kort,"AD/Algemeen Dagblad, Economie; Blz. 16, 329 words",https://advance.lexis.com/api/document?collect...,AD/Algemeen Dagblad,Economie,329,snelle opvolging\n\nNieuwe voorzitter zuivelre...,14-10-2021,kort snelle opvolging nieuwe voorzitter zuivel...,9,1.000000,Environment


In [8]:
# Check for duplicates and count them
duplicates = final_dutch_samples_corrected.duplicated()
num_duplicates = duplicates.sum()

print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


In [9]:
# Save the final samples to an Excel file
final_dutch_samples_corrected.to_excel('1000_Random_Articles.xlsx', index=False)